In [136]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

# Attestation Datasets
att_full1 = pd.read_csv("data/att_full1.csv")
att_full2 = pd.read_csv("data/att_full2.csv")
att_full3 = pd.read_csv("data/att_full3.csv")
att_full4 = pd.read_csv("data/att_full4.csv")
att_full5 = pd.read_csv("data/att_full5.csv")

att_trim1 = pd.read_csv("data/att_trim1.csv")
att_trim2 = pd.read_csv("data/att_trim2.csv")
att_trim3 = pd.read_csv("data/att_trim3.csv")
att_trim4 = pd.read_csv("data/att_trim4.csv")
att_trim5 = pd.read_csv("data/att_trim5.csv")

# Tiny-AES Datasets
aes1 = pd.read_csv("data/tinyaes1.csv")
aes2 = pd.read_csv("data/tinyaes2.csv")
aes3 = pd.read_csv("data/tinyaes3.csv")

# Native
aesn1 = pd.read_csv("data/tinyaesn1.csv")
aesn2 = pd.read_csv("data/tinyaesn2.csv")

# AVL Tree Datasets
# N/A

In [127]:
class Boundary:
    """A boundary is defined by a start and end symbol along with a name."""
    def __init__(self, name, start, end):
        self.name = name
        self.start = start
        self.end = end

In [128]:
"""Given a set of boundaries (in no order) gather the number of cycles it took
to reach the end of each of the boundaries, by returning a pandas dataframe of
the boundaries, their labels, and the cycle cost. This should work in the case
of multiple enclaves being started, running a user/runtime, and then destroying.
"""

def get_indicies(bset, data):
    """Return the indicies of all the found possible boundaries.
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    indicies = []
    for i, row in data.iterrows():
        if row['symbol'] in start_symbols or row['symbol'] in end_symbols:
            if i not in indicies:
                indicies.append(i)
                
    return indicies

def get_cost_table(data):
    """Return table with cycle cost in place of the relative cycle number
    """
    data["end"] = data["symbol"].shift(-1)
    data["cost"] = -(data['cycle'] - data['cycle'].shift(-1))
    # drop the last row because this is cycle cost per instruction
    data = data.drop(['cycle'], axis=1)[:-1]
    return data

def get_symbol_cost_table(phase_df, data):
    """Given a phase cycles table, extract the start and end indicies for each phase, and break
    down those symbols into a pivot table from the original data table.
    """
    # get cost table from original data
    cost_data = get_cost_table(data)
    result = []
    for _, row in phase_df.iterrows():
        start = row['start_index']
        phase = row['phases']
        end = row['end_index']
        symbol_cost_summed = cost_data[start:end].groupby('symbol').sum()
        symbol_cost_summed.reset_index(inplace=True)
        symbol_data = symbol_cost_summed.pivot(columns='symbol', values='cost').bfill().iloc[[0],:]
        symbol_data['phase'] = phase
        result.append(symbol_data)
    # merge dataframes and index on phase
    final = pd.concat(result, ignore_index=True).set_index('phase')
    # sort columns by mean value within column
    return final.reindex(final.mean().sort_values().index, axis=1)

def get_phase_cycles_table(indicies, bset, data):
    """Given indicies of interest and the corresponding data
    return the cycle count of each phase, along with the 
    start and end indicies of each phase which can be used
    for further analysis
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    
    # focus the data
    df = data.iloc[indicies]
    # boundary matching
    phases = []
    cycle_counts = []
    start_indicies = []
    end_indicies = []
    for startidx, row in df.iterrows():
        # continue up until the startidx equals the previous endidx
        if len(end_indicies) > 0:
            if startidx < end_indicies[-1]:
                continue
        start = row['symbol']
        # scan everything in front of the current row
        for endidx, next_row in df.loc[startidx:].iterrows():
            end = next_row['symbol']
            # print(start, end)
            found_end = False
            for b in bset:
                if b.start == start and b.end == end:
                    total_cycles = next_row['cycle'] - row['cycle'] 
                    phases.append(b.name)
                    cycle_counts.append(total_cycles)
                    start_indicies.append(startidx)
                    end_indicies.append(endidx)
                    found_end = True
                    break # on first one, we can stop
            if found_end:
                break
    
    labels = ["phases", "cycle_counts", "start_index", "end_index"]
    res = pd.DataFrame([phases, cycle_counts, start_indicies, end_indicies]).transpose()
    res.columns = labels
    return res

In [129]:
# Smaller bounds should come first.
create = Boundary("create_enclave", "keystone_create_enclave", "keystone_run_enclave")
run_overhead = Boundary("run_enclave", "keystone_run_enclave", "rt_base")

runtime = Boundary("enclave_runtimeuser", "rt_base", "mcall_sm_attest_enclave")
runtime2 = Boundary("enclave_runtimeuser", "return_to_encl", "mcall_sm_stop_enclave")
runtime3 = Boundary("enclave_runtimeuser", "return_to_encl", "mcall_sm_exit_enclave")

attest = Boundary("enclave_attestation", "mcall_sm_attest_enclave", "return_to_encl")
attest_ecall = Boundary("enclave_attestation_edge_call", "mcall_sm_stop_enclave", "return_to_encl")

runtime_noattest = Boundary("enclave_runtimeuser", "rt_base","mcall_sm_stop_enclave")
exit_enclave = Boundary("exit_enclave", "mcall_sm_exit_enclave", "keystone_ioctl")
destroy_bound = Boundary("destroy_enclave", "mcall_sm_destroy_enclave", "enclave_idr_remove")


enclave_bset = [create, run_overhead, runtime, runtime2, runtime3, attest, attest_ecall, runtime_noattest,
               exit_enclave, destroy_bound]

In [130]:
# Run Analysis
def get_buckets(data):
    bset = enclave_bset
    indicies = get_indicies(bset, data)
    # display(data.iloc[indicies]) # sanity check
    phase_table = get_phase_cycles_table(indicies, bset, data)
    # display(phase_table) # sanity check
    results = get_symbol_cost_table(phase_table, data)
    return results.groupby('phase').sum().replace(0, np.nan)

def mean(lst):
    return sum(lst)/len(lst)

def get_error_bounds_per_phase(lst):
    num_phases = lst[0].shape[0]
    
    print("Error bounds:")
    for i in range(num_phases):
        curr_phase_costs = [l.sum(axis=1).values[i] for l in lst]        
        avg = mean(curr_phase_costs)
        mean_sq_diff = [(c - avg)**2 for c in curr_phase_costs]
        avg_mean_sq_diff = mean(mean_sq_diff)
        std_dev = avg_mean_sq_diff**0.5
        err_bound = std_dev / avg
        print(lst[0].index[i] + ":", avg, "std dev:", std_dev, "err:", err_bound * 100)
        
def final_table(lst):
    """Given a lst of bucket tables, get the final result, which is the mean
    of those tables, and sort the indicies by avg value.
    """
    result = mean(lst)
    return result.reindex(result.mean().sort_values(ascending=False).index, axis=1)

In [131]:
t1 = get_buckets(att_trim1)
t2 = get_buckets(att_trim2)
t3 = get_buckets(att_trim3)
t4 = get_buckets(att_trim4)
t5 = get_buckets(att_trim5)

f1 = get_buckets(att_full1)
f2 = get_buckets(att_full2)
f3 = get_buckets(att_full3)
f4 = get_buckets(att_full4)
f5 = get_buckets(att_full5)

a1 = get_buckets(aes1)
a2 = get_buckets(aes2)
a3 = get_buckets(aes3)

atttrimmed = [t1, t2, t3, t4, t5]
attfull = [f1, f2, f3, f4, f5]
aes = [a1, a2, a3]

get_error_bounds_per_phase(atttrimmed)
get_error_bounds_per_phase(attfull)
get_error_bounds_per_phase(aes)

Error bounds:
('create_enclave:', 49494462.2, 'std dev:', 26687.224826871756, 'err:', 0.05391961775245181)
('destroy_enclave:', 240214.4, 'std dev:', 30.017328328817005, 'err:', 0.01249605699276022)
('enclave_attestation:', 1278064.4, 'std dev:', 2218.326269961207, 'err:', 0.17356920902899783)
('enclave_attestation_edge_call:', 4914812.2, 'std dev:', 471.2767339897016, 'err:', 0.009588906245282405)
('enclave_runtimeuser:', 12481.2, 'std dev:', 87.0066664112584, 'err:', 0.69710177235569)
('exit_enclave:', 81846.6, 'std dev:', 1498.771710434915, 'err:', 1.831196055101757)
('run_enclave:', 5212.2, 'std dev:', 172.11670459313353, 'err:', 3.30218918293875)
Error bounds:
('create_enclave:', 49494462.2, 'std dev:', 26687.224826871756, 'err:', 0.05391961775245181)
('destroy_enclave:', 240214.4, 'std dev:', 30.017328328817005, 'err:', 0.01249605699276022)
('enclave_attestation:', 1278064.4, 'std dev:', 2218.326269961207, 'err:', 0.17356920902899783)
('enclave_attestation_edge_call:', 4914812.2,

In [132]:
finalatt= final_table(attfull)
finalatt

,sha3_keccakf,uart_putchar,fe_mul,fe_cmov,fe_sq,memset,init_encl_pgtable,restore_all,hvc_write_room,kmem_cache_alloc,...,osm_pmp_set,hash_finalize,___slab_alloc.isra.10.con,__slab_free.isra.14,bio_attempt_back_merge,cfq_allow_bio_merge,elv_bio_merge_ok,finish_fault,ll_back_merge_fn,put_cpu_partial
phase,,,,,,,,,,,,,,,,,,,,,
create_enclave,48231909.6,NaN,NaN,NaN,NaN,216151.6,67801.0,15496.6,NaN,24459.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
destroy_enclave,NaN,NaN,NaN,NaN,NaN,225871.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_attestation,308692.2,NaN,527170.6,202652.4,155535.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_attestation_edge_call,NaN,4421833.8,NaN,NaN,NaN,5319.4,NaN,156808.2,29580.0,NaN,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_runtimeuser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exit_enclave,NaN,NaN,NaN,NaN,NaN,540.8,NaN,28582.4,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
run_enclave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
finalaes= final_table(aes)
finalaes

,sha3_keccakf,memset,init_encl_pgtable,__copy_user,kmem_cache_alloc,restore_all,hash_epm,__update_load_avg_cfs_rq.,_mix_pool_bytes,_raw_spin_unlock_irqresto,...,swake_up,get_enclave_by_id,work_pending,hash_extend,osm_pmp_set,hash_finalize,mcall_sm_exit_enclave,___slab_alloc.isra.10.con,__slab_free.isra.14,put_cpu_partial
phase,,,,,,,,,,,,,,,,,,,,,
create_enclave,4.823982e+07,216673.0,67819.666667,78413.333333,26628.333333,9705.333333,22992.333333,22969.666667,13467.666667,22401.000000,...,12.0,NaN,6.333333,6.333333,NaN,2.0,NaN,NaN,NaN,NaN
destroy_enclave,NaN,225764.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exit_enclave,NaN,658.0,NaN,683.333333,NaN,42542.666667,NaN,NaN,NaN,1798.333333,...,NaN,NaN,9.000000,NaN,6.0,NaN,2.0,NaN,NaN,NaN
run_enclave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN


In [153]:
def native_cost(df):
    return df.iloc[-1].values[0] - df.iloc[0].values[0]

aes_nat = [aesn1, aesn2]
ncost_aes = [native_cost(a) for a in aes_nat]
navg = mean(ncost_aes)
nstd = np.std(ncost_aes)
print(navg, nstd/eavg * 100)

(206904, 0.04955480093899995)


In [152]:
# aes1[aes1["env"] == "user"]
# print(aes2[aes2["env"] == "user"]["cost"].sum())
# print(aes3[aes3["env"] == "user"]["cost"].sum())
def enclave_app_cost(df):
    ct = get_cost_table(df)
    cost = ct[ct["env"] == "user"]["cost"].sum()
    return cost

aes_enc = [aes1, aes2, aes3]
ecost_aes = [enclave_app_cost(a) for a in aes_enc]
eavg = mean(ecost_aes)
estd = np.std(ecost_aes)
print(eavg, estd/eavg * 100)

(209868.66666666666, 0.07288075954142975)


In [160]:
d = {'Average Runtime Cycles': [navg, eavg], 'Error Bound': [nstd/eavg * 100, estd/eavg * 100]}
chart = pd.DataFrame(data=d, index=["Native", "Enclave"]).round(2)
print(chart.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  Average Runtime Cycles &  Error Bound \\
\midrule
Native  &               206904.00 &         0.05 \\
Enclave &               209868.67 &         0.07 \\
\bottomrule
\end{tabular}



In [120]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import cufflinks as cf

# You'll need to set API key using:
# plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')
# Find your api_key here: https://plot.ly/settings/api
# Cufflinks binds plotly to pandas dataframes in IPython notebook.

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

finalatt.iplot(kind='bar', barmode="stack", filename='name.html')

In [121]:
finalaes.iplot(kind='bar', barmode="stack", filename='name2.html')

In [83]:
# analyze pain points
# for all the phases, get the top offenders
def get_top_x(x, df):
    num_phases = df.shape[0]
    phase_labels = []
    results = []
    for i in range(num_phases):
        phase_data = df.iloc[[i]].transpose()
        phase = phase_data.columns[0]
        phase_data = phase_data[phase_data[phase].notnull()]
        phase_data.columns=["avg_cycles"]
        phase_data["total_percent"] = phase_data["avg_cycles"]/phase_data["avg_cycles"].sum() * 100
        phase_labels.append(phase)
        results.append(phase_data.sort_values("avg_cycles", ascending=False)[:x])
    return phase_labels, results

In [84]:
att_labels, att_results = get_top_x(15, finalatt)

In [85]:
aes_labels, aes_results = get_top_x(15, finalaes)

In [108]:
print(att_labels)
for r in att_results:
    print(r.round(2).to_latex())

['create_enclave', 'destroy_enclave', 'enclave_attestation', 'enclave_attestation_edge_call', 'enclave_runtimeuser', 'exit_enclave', 'run_enclave']
\begin{tabular}{lrr}
\toprule
{} &  avg\_cycles &  total\_percent \\
\midrule
sha3\_keccakf              &  48231909.6 &          97.45 \\
memset                    &    216151.6 &           0.44 \\
\_\_copy\_user               &     68160.8 &           0.14 \\
init\_encl\_pgtable         &     67801.0 &           0.14 \\
restore\_regs              &     34903.2 &           0.07 \\
\_\_cpuidle\_text\_end        &     30753.8 &           0.06 \\
kmem\_cache\_alloc          &     24459.0 &           0.05 \\
\_\_update\_load\_avg\_cfs\_rq. &     23483.8 &           0.05 \\
hash\_epm                  &     23000.4 &           0.05 \\
\_raw\_spin\_unlock\_irqresto &     22608.8 &           0.05 \\
\_raw\_spin\_lock\_irqsave    &     20469.8 &           0.04 \\
restore\_all               &     15496.6 &           0.03 \\
kfree                    

In [122]:
print(aes_labels)
for r in aes_results:
    print(r.round(2).to_latex())

['create_enclave', 'destroy_enclave', 'exit_enclave', 'run_enclave']
\begin{tabular}{lrr}
\toprule
{} &   avg\_cycles &  total\_percent \\
\midrule
sha3\_keccakf              &  48239823.67 &          97.42 \\
memset                    &    216673.00 &           0.44 \\
\_\_copy\_user               &     78413.33 &           0.16 \\
init\_encl\_pgtable         &     67819.67 &           0.14 \\
restore\_regs              &     34699.67 &           0.07 \\
\_\_cpuidle\_text\_end        &     30335.67 &           0.06 \\
kmem\_cache\_alloc          &     26628.33 &           0.05 \\
hash\_epm                  &     22992.33 &           0.05 \\
\_\_update\_load\_avg\_cfs\_rq. &     22969.67 &           0.05 \\
\_raw\_spin\_unlock\_irqresto &     22401.00 &           0.05 \\
\_raw\_spin\_lock\_irqsave    &     19985.33 &           0.04 \\
kfree                     &     14988.00 &           0.03 \\
\_raw\_spin\_lock\_irq        &     14636.00 &           0.03 \\
trap\_vector               

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd
import numpy as np

N = 20
x = np.linspace(1, 10, N)
y = np.random.randn(N)+3
y2 = np.random.randn(N)+6
y3 = np.random.randn(N)+9
y4 = np.random.randn(N)+12
df = pd.DataFrame({'x': x, 'y': y, 'y2':y2, 'y3':y3, 'y4':y4})
df.head()

data = [
    go.Bar(
        x=df['x'], # assign x as the dataframe column 'x'
        y=df['y']
    ),
    go.Bar(
        x=df['x'],
        y=df['y2']
    ),
    go.Bar(
        x=df['x'],
        y=df['y3']
    ),
    go.Bar(
        x=df['x'],
        y=df['y4']
    )

]

layout = go.Layout(
    barmode='stack',
    title='Stacked Bar with Pandas'
)

fig = go.Figure(data=data, layout=layout)

# IPython notebook
py.iplot(fig, filename='pandas-bar-chart-layout')

In [ ]:
# Some notes
# IDEA: Plot top 15 in a pie chart for each binary in each phase (attestation, tiny_aes, max element in array)
# TODO: Remove uart_putchar
# TODO: Automate tracefile generation and symbol analysis